In [3]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time

def get_jobs(keyword, num_jobs, verbose=False):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    
    # Uncomment to run headless
    # options.add_argument("--headless")
    
    service = Service("C:/Users/saudq/Desktop/MachineLearning/Salary project/chromedriver.exe")  # Change this path
    driver = webdriver.Chrome(service=service, options=options)
    
    url = f'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="{keyword}"'
    driver.get(url)
    jobs = []
    
    while len(jobs) < num_jobs:
        time.sleep(4)  # Adjust according to internet speed
        
        try:
            close_button = driver.find_element(By.CLASS_NAME, "ModalStyle__xBtn___29PT9")
            close_button.click()
        except NoSuchElementException:
            pass
        
        job_buttons = driver.find_elements(By.CLASS_NAME, "jobListing")
        for job_button in job_buttons:
            if len(jobs) >= num_jobs:
                break
            
            job_button.click()
            time.sleep(2)
            
            try:
                company_name = driver.find_element(By.CLASS_NAME, "employerName").text
                location = driver.find_element(By.CLASS_NAME, "location").text
                job_title = driver.find_element(By.CLASS_NAME, "jobTitle").text
                job_description = driver.find_element(By.CLASS_NAME, "jobDescriptionContent").text
            except NoSuchElementException:
                continue
            
            try:
                salary_estimate = driver.find_element(By.CLASS_NAME, "salary").text
            except NoSuchElementException:
                salary_estimate = "Not Available"
            
            try:
                rating = driver.find_element(By.CLASS_NAME, "rating").text
            except NoSuchElementException:
                rating = "Not Available"
            
            if verbose:
                print(f"Job Title: {job_title}\nSalary Estimate: {salary_estimate}\nRating: {rating}\nCompany Name: {company_name}\nLocation: {location}\n")
            
            jobs.append({
                "Job Title": job_title,
                "Salary Estimate": salary_estimate,
                "Job Description": job_description,
                "Rating": rating,
                "Company Name": company_name,
                "Location": location
            })
            
        try:
            next_button = driver.find_element(By.XPATH, '//li[@class="next"]//a')
            next_button.click()
        except NoSuchElementException:
            print(f"Scraping stopped before reaching target. Needed {num_jobs}, got {len(jobs)}.")
            break
    
    driver.quit()
    return pd.DataFrame(jobs)

In [15]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 5, False)
df

Scraping stopped before reaching target. Needed 5, got 0.


""
